In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
prev_app = pd.read_csv('previous_application.csv')

In [3]:
prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
prev_app.shape

(1670214, 37)

In [5]:
prev_app = pd.get_dummies(prev_app)

In [6]:
prev_app.shape

(1670214, 164)

In [7]:
# prev_app['DAYS_FIRST_DRAWING'] = prev_app['DAYS_FIRST_DRAWING'].replace(365243, np.nan)

In [8]:
curr_app = pd.read_csv('application_train.csv')

In [9]:
y = curr_app['TARGET']

In [10]:
X = curr_app.iloc[:, :1]

In [11]:
X_test = pd.read_csv('application_test.csv')
X_test = X_test.iloc[:, :1]

In [12]:
counts = prev_app.groupby('SK_ID_CURR').size()

In [13]:
gb_sum_df = prev_app.groupby('SK_ID_CURR').sum().reset_index().drop(columns=['SK_ID_PREV'])

In [14]:
gb_sum_df.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,...,PRODUCT_COMBINATION_Cash X-Sell: low,PRODUCT_COMBINATION_Cash X-Sell: middle,PRODUCT_COMBINATION_POS household with interest,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest
0,100001,3951.000,24835.5,23787.0,2520.0,24835.5,13,1,0.104326,0.0,...,0,0,0,0,0,0,1,0,0,0
1,100002,9251.775,179055.0,179055.0,0.0,179055.0,9,1,0.000000,0.0,...,0,0,0,0,0,0,0,0,1,0
2,100003,169661.970,1306309.5,1452573.0,6885.0,1306309.5,44,3,0.100061,0.0,...,1,0,1,0,1,0,0,0,0,0
3,100004,5357.250,24282.0,20106.0,4860.0,24282.0,5,1,0.212008,0.0,...,0,0,0,0,0,0,0,1,0,0
4,100005,4813.200,44617.5,40153.5,4464.0,44617.5,21,2,0.108964,0.0,...,0,0,0,0,0,0,1,0,0,0


In [15]:
gb_sum_df['TERM'] = gb_sum_df['AMT_CREDIT'] /gb_sum_df['AMT_ANNUITY']

In [16]:
gb_sum_df['TERM'] = gb_sum_df['TERM'].replace(np.inf, np.nan)

In [17]:
gb_sum_df['COUNTS'] = counts

In [18]:
gb_sum_df['RATE_DOWN_PAYMENT_AVG'] = gb_sum_df['RATE_DOWN_PAYMENT'] / gb_sum_df['COUNTS']

In [19]:
X = X.merge(gb_sum_df, on='SK_ID_CURR', how='left')
X_test = X_test.merge(gb_sum_df, on='SK_ID_CURR', how='left')

In [20]:
X.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,HOUR_APPR_PROCESS_START,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,...,PRODUCT_COMBINATION_POS household without interest,PRODUCT_COMBINATION_POS industry with interest,PRODUCT_COMBINATION_POS industry without interest,PRODUCT_COMBINATION_POS mobile with interest,PRODUCT_COMBINATION_POS mobile without interest,PRODUCT_COMBINATION_POS other with interest,PRODUCT_COMBINATION_POS others without interest,TERM,COUNTS,RATE_DOWN_PAYMENT_AVG
0,100002,9251.775,179055.00,179055.0,0.00,179055.00,9.0,1.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,19.353584,NaN,NaN
1,100003,169661.970,1306309.50,1452573.0,6885.00,1306309.50,44.0,3.0,0.100061,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,8.561571,NaN,NaN
2,100004,5357.250,24282.00,20106.0,4860.00,24282.00,5.0,1.0,0.212008,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.753045,NaN,NaN
3,100006,141907.050,2449829.34,2625259.5,69680.34,2449829.34,132.0,9.0,0.326824,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,18.499853,NaN,NaN
4,100007,73672.830,903181.50,999832.5,6781.50,903181.50,74.0,6.0,0.319033,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,13.571251,NaN,NaN


In [21]:
corrs = pd.Series([y.corr(X[col]) for col in X.columns[-gb_sum_df.shape[1]+1:]], index=gb_sum_df.columns[1:]).sort_values()

In [22]:
corrs = abs(corrs).sort_values(ascending=False)
corrs.head(50)

NAME_CONTRACT_STATUS_Refused                          0.064469
NAME_PRODUCT_TYPE_walk-in                             0.062628
CODE_REJECT_REASON_SCOFR                              0.056284
PRODUCT_COMBINATION_Card Street                       0.047953
CODE_REJECT_REASON_HC                                 0.047067
NAME_CLIENT_TYPE_New                                  0.046048
NAME_CONTRACT_TYPE_Revolving loans                    0.045602
NAME_YIELD_GROUP_XNA                                  0.041713
RATE_DOWN_PAYMENT                                     0.041693
DAYS_FIRST_DRAWING                                    0.041671
NAME_PORTFOLIO_Cards                                  0.041480
CODE_REJECT_REASON_LIMIT                              0.040503
NAME_PAYMENT_TYPE_XNA                                 0.039469
NAME_YIELD_GROUP_high                                 0.036931
PRODUCT_COMBINATION_Cash X-Sell: high                 0.036291
CHANNEL_TYPE_AP+ (Cash loan)                          0

In [23]:
X[corrs.index].to_pickle('features_prev.pkl')
X_test[corrs.index].to_pickle('features_prev_test.pkl')